---
## SQL Agents
Fourth approach, continued from SQLBasedChains.ipynb.

Agent created from *create_sql_agent*, contains *SQLDatabaseToolkit*.
* It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table)
* It can recover from errors by running a generated query, catching the traceback and regenerating it correctly


In [2]:
from langchain.agents.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.sql_database import SQLDatabase
from langchain.llms import GPT4All


memory = ConversationBufferMemory(memory_key = 'history' , input_key = 'input')
llm = GPT4All(model="../models/ggml-model-gpt4all-falcon-q4_0.bin")
db = SQLDatabase.from_uri("sqlite:///../db/retention-sqlite.db")


suffix = """Begin!

Relevant pieces of previous conversation:
{history}
(You do not need to use these pieces of information if not relevant)

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}
"""
executor = create_sql_agent(
    llm = llm,
    toolkit = SQLDatabaseToolkit(db=db, llm=llm),
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    input_variables = ["input", "agent_scratchpad", "history"],
    suffix = suffix, # must have history as variable,
    agent_executor_kwargs = {'memory':memory},
    verbose=True,
)

Found model file at  ../models/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from '../models/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: 

objc[62078]: Class GGMLMetalClass is implemented in both /opt/miniconda3/envs/gnn/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x127180228) and /opt/miniconda3/envs/gnn/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x127298228). One of the two will be used. Which one is undefined.


........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [3]:
executor.run("What are the tables in the database?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `
Action: Look at the tables in the database using sql_db_schema`